# Dinámica de transición de estados

Sea un sistema de tres estados bombeado por una fuente de intensidad de potencia constante $P$. Si el haz de excitación se considera Gaussiano, se puede calcular una tasa de bombeo dada por $$\rho_p=\frac{\lambda_p}{hc\pi \omega_p²} P.$$ Con lo cual la tasa de excitación viene dada por$$R_i =\rho_p\sigma_iN_i,$$ donde $\sigma_i$ es la sección eficaz de absorción del estado $i$ y $N_i$ su nivel de ocupación.
En este caso el coeficiente de absorción total para este bombeo vendrá dado por la suma de la absorción de todas las bandas

$$\alpha=\sum_{j=1}^{n-1}\sigma_jN_j.$$

En un sistema donde puede ocurrir GSA, ETU y ESA se pueden escribir las
siguientes ecuaciones dinámicas:

$$\dot N_1=\rho_p\sigma_0N_0-2k_{\text{ETU}}N_1²-\rho_p\sigma_1N_1-k_1N_1\ ,$$
$$\dot N_2=\rho_p\sigma_1N_1+k_{\text{ETU}}N_1²-k_2N_2\ .$$



*   Nótese que en la segunda ec., el factor 2 implica que se requieren dos iones de el nivel $N_1$ para generar uno en $N_2$, y es esto lo que agrega el comportamiento no lineal al sistema de ecs. de un sistema con ETU.
*   Los términos que incluyen a $\sigma_1$ responden a la sobreexcitación del estado $N_1$, es decir que son términos correspondientes a ESA.
Los decaimientos lineales nivel base vienen dados por tasas $k_1$ y $k_2$.
* Comportamiento estacionario: $\dot N_1=\dot N_2=0$. En función de la potencia de excitación, las pendientes límite de las funciones $N_1(P)$ y $N_2(P)$ refieren a algunas variables internas del proceso.




## Caso ETU dominante:

$$\rho_p\sigma_0N_0=2k_{\text{ETU}}N_1²+k_1N_1\ ,$$
$$k_{\text{ETU}}N_1²=k_2N_2\ .$$

* $N_2\propto N_1²$
* $k_1\gg k_{\text{ETU}}\Rightarrow N_2\propto N_1²\propto P²$.
* $k_1\ll k_{\text{ETU}}\Rightarrow N_2\propto P$.

## Caso ESA dominante:

$$\rho_p\sigma_0N_0=\rho_p\sigma_1N_1+k_1N_1\ ,$$
$$\rho_p\sigma_1N_1=k_2N_2\ .$$

* $N_2\propto PN_1$
* $k_1\gg \rho_p\sigma_1\Rightarrow N_2\propto P²$.
* $k_1\ll \rho_p\sigma_1\Rightarrow N_2\propto P$ (y $N_1$ es independiente de la potencia de excitación).

Una simulación de estos dos procesos debería evidenciar que la observación del estado $N_2$ no permite establecer una conclusión sobre cual es el proceso preponderante, pero $N_1$ sí permite distinguir claramente cada uno de los casos.

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import ipywidgets as widgets
from ipywidgets import interactive

## Caso ETU dominante:

In [16]:
# Definir el sistema de ecuaciones
def system(vars, rho_p, N_0, k_ETU, k_1, k_2):
    N_1, N_2 = vars
    eq1 = rho_p * N_0 - (2 * k_ETU * N_1**2 + k_1 * N_1)
    eq2 = k_ETU * N_1**2 - k_2 * N_2
    return [eq1, eq2]

# Función para resolver el sistema y graficar los resultados
def plot_system(N_0, k_ETU, k_1, k_2):
    # Crear los valores de rho_p en escala logarítmica
    rho_p_values = np.logspace(-1, 5, 100)  # 100 puntos entre 10^-3 y 10^4
    N_1_values = []
    N_2_values = []

    # Resolver el sistema para cada valor de rho_p
    for rho_p in rho_p_values:
        initial_guess = [1, 1]
        solution = fsolve(system, initial_guess, args=(rho_p, N_0, k_ETU, k_1, k_2))
        N_1, N_2 = solution
        N_1_values.append(N_1)
        N_2_values.append(N_2)

    # Graficar los resultados en escala log-log
    plt.figure(figsize=(8, 6))
    plt.loglog(rho_p_values, N_1_values, label=r'$N_1$', color='k')
    plt.loglog(rho_p_values, N_2_values, label=r'$N_2$', color='r')
    plt.xlabel(r'$\rho_p$', fontsize=14)
    plt.ylabel(r'$N_1$, $N_2$', fontsize=14)
    plt.title(f'Solución del sistema para N_0={N_0}, k_ETU={k_ETU:.0e}, k_1={k_1:.0e}, k_2={k_2:.0e}')
    plt.legend()
    plt.grid(True, which="both", alpha=.25)
    plt.show()

# Crear los sliders con los nuevos valores y rangos
# N_0 (de 0 a 1000, con 100 pasos)
N_0_slider = widgets.IntSlider(value=500, min=0, max=1000, step=100, description='N_0', continuous_update=False)

# Crear valores logarítmicos para k_ETU, k_1, k_2 entre 10^0 y 10^5 (10 valores discretos)
log_values = np.logspace(0, 5, 10)
lin_values = np.arange(1E10,1E15+1,1E12)

# Encontrar el valor más cercano a 1000 en la lista de log_values
def closest_value(value, options):
    return min(options, key=lambda x: abs(x - value))

# Establecer valores iniciales usando el valor más cercano
k_ETU_initial = closest_value(1E10, lin_values)
k_1_initial = closest_value(1E10, lin_values)
k_2_initial = closest_value(1E10, lin_values)

# Crear los sliders para k_ETU, k_1 y k_2
k_ETU_slider = widgets.SelectionSlider(
    options=lin_values,
    value=k_ETU_initial,
    description=f'k_ETU = {k_ETU_initial:.2e}',
    continuous_update=False
)
k_1_slider = widgets.SelectionSlider(
    options=lin_values,
    value=k_1_initial,
    description=f'k_1 = {k_1_initial:.2e}',
    continuous_update=False
)
k_2_slider = widgets.SelectionSlider(
    options=lin_values,
    value=k_2_initial,
    description=f'k_2 = {k_2_initial:.2e}',
    continuous_update=False
)
# Crear la interfaz interactiva
interactive_plot = interactive(plot_system,
                               N_0=N_0_slider,
                               k_ETU=k_ETU_slider,
                               k_1=k_1_slider,
                               k_2=k_2_slider)

# Mostrar los sliders y la gráfica
display(interactive_plot)

interactive(children=(IntSlider(value=500, continuous_update=False, description='N_0', max=1000, step=100), Se…

In [17]:
import ipywidgets as widgets
widgets.Widget.widgets = {}

# backup ------------------------

In [18]:
# Definir el sistema de ecuaciones
def system(vars, rho_p, N_0, k_ETU, k_1, k_2):
    N_1, N_2 = vars
    eq1 = rho_p * N_0 - (2 * k_ETU * N_1**2 + k_1 * N_1)
    eq2 = k_ETU * N_1**2 - k_2 * N_2
    return [eq1, eq2]

# Función para resolver el sistema y graficar los resultados
def plot_system(N_0, k_ETU, k_1, k_2):
    # Crear los valores de rho_p en escala logarítmica
    rho_p_values = np.logspace(-1, 5, 100)  # 100 puntos entre 10^-3 y 10^4
    N_1_values = []
    N_2_values = []

    # Resolver el sistema para cada valor de rho_p
    for rho_p in rho_p_values:
        initial_guess = [1, 1]
        solution = fsolve(system, initial_guess, args=(rho_p, N_0, k_ETU, k_1, k_2))
        N_1, N_2 = solution
        N_1_values.append(N_1)
        N_2_values.append(N_2)

    # Graficar los resultados en escala log-log
    plt.figure(figsize=(8, 6))
    plt.loglog(rho_p_values, N_1_values, label=r'$N_1$', color='k')
    plt.loglog(rho_p_values, N_2_values, label=r'$N_2$', color='r')
    plt.xlabel(r'$\rho_p$', fontsize=14)
    plt.ylabel(r'$N_1$, $N_2$', fontsize=14)
    plt.title(f'Solución del sistema para N_0={N_0}, k_ETU={k_ETU:.0e}, k_1={k_1:.0e}, k_2={k_2:.0e}')
    plt.legend()
    plt.grid(True, which="both", alpha=.25)
    plt.show()

# Crear los sliders con los nuevos valores y rangos
# N_0 (de 0 a 1000, con 100 pasos)
N_0_slider = widgets.IntSlider(value=500, min=0, max=1000, step=100, description='N_0', continuous_update=False)

# Crear valores logarítmicos para k_ETU, k_1, k_2 entre 10^0 y 10^5 (10 valores discretos)
log_values = np.logspace(0, 5, 10)
lin_values = np.arange(1E10,1E15+1,1E12)

# Encontrar el valor más cercano a 1000 en la lista de log_values
def closest_value(value, options):
    return min(options, key=lambda x: abs(x - value))

# Establecer valores iniciales usando el valor más cercano
k_ETU_initial = closest_value(1E10, lin_values)
k_1_initial = closest_value(1E10, lin_values)
k_2_initial = closest_value(1E10, lin_values)

# Crear los sliders para k_ETU, k_1 y k_2
k_ETU_slider = widgets.SelectionSlider(
    options=lin_values,
    value=k_ETU_initial,
    description=f'k_ETU = {k_ETU_initial:.2e}',
    continuous_update=False
)
k_1_slider = widgets.SelectionSlider(
    options=lin_values,
    value=k_1_initial,
    description=f'k_1 = {k_1_initial:.2e}',
    continuous_update=False
)
k_2_slider = widgets.SelectionSlider(
    options=lin_values,
    value=k_2_initial,
    description=f'k_2 = {k_2_initial:.2e}',
    continuous_update=False
)
# Crear la interfaz interactiva
interactive_plot = interactive(plot_system,
                               N_0=N_0_slider,
                               k_ETU=k_ETU_slider,
                               k_1=k_1_slider,
                               k_2=k_2_slider)

# Mostrar los sliders y la gráfica
display(interactive_plot)

interactive(children=(IntSlider(value=500, continuous_update=False, description='N_0', max=1000, step=100), Se…

In [19]:
import ipywidgets as widgets
widgets.Widget.widgets = {}